In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import os
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Define NYC MTA turnstile data URL
url = 'http://web.mta.info/developers/turnstile.html'

In [3]:
# Send request and verify URL
rsp = requests.get(url)
print(rsp.url)

http://web.mta.info/developers/turnstile.html


In [4]:
# Parse HTML response
soup = BeautifulSoup(rsp.text, 'html.parser')

In [5]:
# Find all anchor tags with href attribute defined
a_tags = soup.find_all('a', href=True)

In [6]:
file_list = []
base_url = url.split('turnstile')[0]
for a in a_tags:
    tmp = re.search('data/nyct/turnstile/turnstile_\d*\.txt', a['href'])
    if tmp is not None:
        file_list.append(''.join([base_url, tmp.string]))        

In [7]:
# Download txt files from NYC MTA website to local machine
# for file in file_list:
#     !wget -P ./turnstile_data/ {file}

In [8]:
# Build DF
data_path = '/home/cneiderer/Metis/turnstile_data' # path to directory with files
df = pd.DataFrame()
file_names = os.listdir(data_path)
for file in file_names[:2]:
    file_path = os.path.join(data_path, file)
    df = pd.concat([df, pd.read_csv(file_path)], axis=0)  
# Reset index for concatenated dataframe    
df = df.reset_index(drop=True)   
# Remove whitespace from column names and make lowercase
df.columns = [x.strip().lower() for x in df.columns]

In [9]:
df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits
0,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,00:00:00,REGULAR,5600838,1896290
1,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,04:00:00,REGULAR,5600863,1896293
2,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,08:00:00,REGULAR,5600877,1896313
3,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,12:00:00,REGULAR,5600922,1896375
4,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,16:00:00,REGULAR,5601172,1896445


In [10]:
# Calculate count differences between audits and add to df
df[['entries_diff', 'exits_diff']] = df.groupby('station')[['entries', 'exits']].diff().fillna(0)

In [11]:
df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,entries_diff,exits_diff
0,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,00:00:00,REGULAR,5600838,1896290,0.0,0.0
1,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,04:00:00,REGULAR,5600863,1896293,25.0,3.0
2,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,08:00:00,REGULAR,5600877,1896313,14.0,20.0
3,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,12:00:00,REGULAR,5600922,1896375,45.0,62.0
4,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,16:00:00,REGULAR,5601172,1896445,250.0,70.0


In [12]:
# Conver date and time strings to datetime objects
df['date_time'] = pd.to_datetime(df.date + df.time, format='%m/%d/%Y%H:%M:%S')
#df['date_obj'] = pd.to_datetime(df.date, format='%m/%d/%Y')
#df['time_obj'] = pd.to_datetime(df.time, format='%H:%M:%S')

In [13]:
df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits,entries_diff,exits_diff,date_time
0,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,00:00:00,REGULAR,5600838,1896290,0.0,0.0,2016-04-02 00:00:00
1,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,04:00:00,REGULAR,5600863,1896293,25.0,3.0,2016-04-02 04:00:00
2,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,08:00:00,REGULAR,5600877,1896313,14.0,20.0,2016-04-02 08:00:00
3,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,12:00:00,REGULAR,5600922,1896375,45.0,62.0,2016-04-02 12:00:00
4,A002,R051,02-00-00,59 ST,NQR456,BMT,04/02/2016,16:00:00,REGULAR,5601172,1896445,250.0,70.0,2016-04-02 16:00:00


### Sandbox Below

In [14]:
df2[datetime.datetime(1900, 1, 1, 8, 0, 0, 0) == df2.time_obj]

NameError: name 'df2' is not defined

In [ ]:
df2.time_obj.diff().fillna(0)

In [ ]:
df2.date_time.diff().fillna(0)

In [ ]:
t2 = df2.time[1:].reset_index(drop=True).

In [ ]:
t1 = df2.time[0:-1].reset_index(drop=True)

In [ ]:
pd.concat([t1.rename('t1'), t2.rename('t2')], axis=1)